In [3]:
import pandas as pd
import numpy as np

In [4]:
y_true = pd.DataFrame([[0, 0, 1],
                      [0, 1, 0],
                      [0, 1, 1],
                      [1, 1, 0],
                      [1, 1, 1]],
                      columns = ["tree", "grass", "flower"])

In [1]:
def dataframe_baseline_creator(df):

    for column in df.columns:
        
        df[column] = df[column].sum()/len(df)
    
    return df
    
    

In [5]:
dataframe_baseline_creator(y_true)

,tree,grass,flower
0,0.4,0.8,0.6
1,0.4,0.8,0.6
2,0.4,0.8,0.6
3,0.4,0.8,0.6
4,0.4,0.8,0.6
